Данное решение было оценено в 65.1. Оно представляет из себя тривиальное предсказание с проверкой преобразования в элементы орбиты.

In [1]:
from coosys import cartesian_to_kepler as ctk, cartesian_to_quaternion as ctq
from coosys import kepler_to_cartesian as ktc, quaternion_to_cartesian as qtc
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import PolynomialFeatures as PF
from tqdm.auto import tqdm
tqdm.pandas()

from matplotlib import pyplot as plt
%matplotlib inline

D:\Anaconda3\envs\Standard\lib\site-packages\tqdm\std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
base = 1.3885 * 10**9

train_data = pd.read_csv("IDAO 2020/train.csv", encoding="utf8")
train_data["epoch"] = pd.to_datetime(train_data["epoch"]).apply(pd.Timestamp.timestamp) - base

test_data = pd.read_csv("IDAO 2020/Track 1/test.csv", encoding="utf8")
test_data["epoch"] = pd.to_datetime(test_data["epoch"]).apply(pd.Timestamp.timestamp) - base

In [3]:
sats_to_predict = set(test_data["sat_id"].unique())

In [4]:
result = test_data[["id", "x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"]].rename(columns={
    "x_sim": "x",
    "y_sim": "y",
    "z_sim": "z",
    "Vx_sim": "Vx",
    "Vy_sim": "Vy",
    "Vz_sim": "Vz",
})

In [5]:
gamma_km = 398603

def transform_row(row, func):
    return func(row.values)

def transform_dataset(data, func=lambda data: ctk(data, gamma_km), 
                      columns_from=("x", "y", "z", "Vx", "Vy", "Vz"), 
                      columns_to=("a", "e", "inclination", "longitude", "argument", "anomaly")):
    new_data = data.loc[:, list(columns_from)].progress_apply(transform_row, axis=1, result_type="expand", args=(func, ))
    new_data.columns = columns_to
    return data.drop(list(columns_from), axis=1).join(new_data)

In [6]:
result = transform_dataset(result, lambda data: ctk(data, gamma_km),
                           columns_from=("x", "y", "z", "Vx", "Vy", "Vz"), 
                           columns_to=("a", "e", "inclination", "longitude", "argument", "anomaly"))
result = transform_dataset(result, lambda data: ktc(data, gamma_km),
                           columns_from=("a", "e", "inclination", "longitude", "argument", "anomaly"), 
                           columns_to=("x", "y", "z", "Vx", "Vy", "Vz"))

In [7]:
result.to_csv("submission.csv", index=False)